In [1]:
suppressMessages(library("rwwa"))

In [2]:
# load covariate data
df <- merge(load_ts("EA-drought_gmst-corrected.dat", col.names = c("year", "gmst")),
            load_ts("../ts/EA-drought_detrended-nino-ond.dat", col.names = c("year", "nino_ond")))

# fix covariate values
gmst_2022 <- df[df$year == 2022, "gmst"]
gmst_cf <- gmst_2022 - 1.2

ondnino_2022 <- ondnino_cf <- df[df$year == 2022, "nino_ond"]

In [3]:
# factual & counterfactual climates
cov_2022 <- df[df$year == 2022,c("gmst", "nino_ond")]

cov_cf <- rbind("pi" = cov_2022 - c(1.2, 0),
                "nino" = c(cov_2022$gmst, 0)) 

# Check trend fitting

In [6]:
png_res = 300

invisible(sapply(c("mam", "ond", "24"), function(seas) {
    
    if(seas == "ond") { covnm = c("gmst", "nino_ond") } else { covnm = c("gmst") }      # OND needs second covariate
    
    # loop over all available datasets, get results for each
    res <- do.call("rbind", sapply(c("cpc", "chirps05+centrends01"), function(dsnm) {
               
        pr_df <- merge(df, load_ts(paste0("../ts/EA-drought_pr-", seas,"_",dsnm,".dat"), col.names = c("year", "pr")))
        pr_df["log10_pr"] <- log10(pr_df$pr)
        
        mdl <- fit_ns("norm", "fixeddisp", pr_df, varnm = "log10_pr", covnm = covnm, lower = T)

        # plot fitted model
        png(paste0("res/fig_pr-",seas,"_",dsnm,".png"), h = png_res, w = png_res * 3); {
            prep_window(c(1,3))
        
            plot_trend(mdl)
            plot_covtrend(mdl, "gmst")
            plot_returnlevels(mdl, cov_f = cov_2022, cov_cf = cov_cf["pi",,drop = F], nsamp = 500)
            mtext(side = 3, dsnm, outer = T, line = -2, font = 2)
        }; dev.off()

        # bootstrap results & save
        boot_res <- boot_ci(mdl, cov_f = cov_2022, cov_cf = cov_cf, nsamp = 1000, seed = 101)
        boot_res["dI_rel_pi",] <- ((10**boot_res["dI_abs_pi",]) - 1)*100            # calculate relative change
        unlist(lapply(rownames(boot_res), function(cnm) setNames(boot_res[cnm,], paste(gsub("_", "-", cnm), c("est", "lower", "upper"), sep = "_"))))
        
    }, simplify = F))
    write.csv(res, paste0("res/EA-drought_pr-", seas, "_obs_fitted.csv"))
}))

## Table of results

In [ ]:
seas <- "24"
res_org <- read.csv(paste0("../res/EA-drought_pr-",seas,"_obs_fitted.csv"), row.names = "X")[1:2,]
res_upd <- read.csv(paste0("res/EA-drought_pr-",seas,"_obs_fitted.csv"), row.names = "X")

res <- list("dI_org" = res_org[,grepl("rel_DI", colnames(res_org))],
            "dI_upd" = res_upd[,grepl("dI.rel.pi", colnames(res_upd))],
            "pr_org" = res_org[,grepl("pr", colnames(res_org))],
            "pr_upd" = res_upd[,grepl("PR.pi", colnames(res_upd))],
            "rp_org" = res_org[,grepl("rp", colnames(res_org))],
            "rp_upd" = res_upd[,grepl("return.period", colnames(res_upd))])

sapply(res, function(df) apply(signif(df, 2), 1, function(r) paste0(r[1], " (",r[2], ", ", r[3], ")")))

## Revised figures

In [62]:
for (seas in c("mam", "ond", "24")) {
    
    if(seas == "ond") { covnm = c("gmst", "nino_ond") } else { covnm = c("gmst") }      # OND needs second covariate
    
    # loop over all available datasets, get results for each
    cpc <- load_ts(paste0("../ts/EA-drought_pr-", seas,"_cpc.dat"), col.names = c("year", "pr"))
    cpc["log10_pr"] <- log10(cpc$pr)
    
    ctrends <- load_ts(paste0("../ts/EA-drought_pr-", seas,"_chirps05+centrends01.dat"), col.names = c("year", "pr"))
    ctrends["log10_pr"] <- log10(ctrends$pr)
            
    mdl_cpc <- fit_ns("norm", "fixeddisp", merge(df, cpc), varnm = "log10_pr", covnm = covnm, lower = T)
    mdl_ctrends <- fit_ns("norm", "fixeddisp", merge(df, ctrends), varnm = "log10_pr", covnm = covnm, lower = T)

    ylim <- list("24" = c(2.6, 3.2), "mam" = c(1.8, 2.8), "ond" = c(1.6, 2.8))[[seas]]
    xlim <- c(-0.45, 0.95)
    png_res = 360
    
    # plot fitted model
    png(paste0("res/fig_pr-",seas,".png"), h = png_res*2, w = png_res * 2); {
        prep_window(c(2,2), oma = c(0,1,0.1,0))
    
        plot_covtrend(mdl_cpc, "gmst", xlab = "GMST anomaly", legend_pos = NA, ylab = "log10 precip", ylim = ylim, ci_cov = data.frame(gmst = gmst_2022 - c(0,1.2)), xlim = xlim)
        mtext(side = 3, "CPC", outer = F, line = 1, font = 2, adj = 0, cex = 1.1)
        plot_returnlevels(mdl_cpc, cov_f = cov_2022, cov_cf = cov_cf["pi",,drop = F], ylim = ylim, ylab = "log10 precip", nsamp = 5, model_desc = F, legend_labels = c("2022 GMST", "2022 GMST - 1.2C"))
        
        plot_covtrend(mdl_ctrends, "gmst", xlab = "GMST anomaly", legend_pos = NA, ylab = "log10 precip", ylim = ylim, ci_cov = data.frame(gmst = gmst_2022 - c(0,1.2)), xlim = xlim)
        mtext(side = 3, "CHIRPS + CenTrends ", outer = F, line = 1, font = 2, adj = 0, cex = 1.1)
        plot_returnlevels(mdl_ctrends, cov_f = cov_2022, cov_cf = cov_cf["pi",,drop = F], ylim = ylim, ylab = "log10 precip", nsamp = 5, model_desc = F, legend_labels = c("2022 GMST", "2022 GMST - 1.2C"))

        mtitle = list("24" = "24-month rainfall", "mam" = "MAM rainfall", "ond" = "OND rainfall")[[seas]]
        mtext(side = 2, mtitle, outer = T, line = 0, font = 2, cex = 1.1)
    }; dev.off()
}

# Synthesis

In [4]:
# get list of models to include (copied from spreadsheets)
mlist <- read.csv("mdl-eval_EA-drought.csv")

In [13]:
seas <- "24"

In [14]:
res_obs <- read.csv(paste0("res/EA-drought_pr-",seas,"_obs_fitted.csv"), row.names = "X")
rownames(res_obs) <- c("CPC", "CenTrends + CHIRPS")

res_cordex <- read.csv(paste0("../res/EA-drought_pr-",seas,"_cordex_fitted.csv"), row.names = "X")
res_hiresmip <- read.csv(paste0("../res/EA-drought_pr-",seas,"_highresmip_fitted.csv"), row.names = "X")

In [15]:
include <- mlist[mlist[,paste0("keep_",seas)] == "y","mdl"]

In [16]:
# get required columns from obs results
pr_obs <- res_obs[,grepl("PR.pi", colnames(res_obs))]
dI_obs <- res_obs[,grepl("dI.rel.pi", colnames(res_obs))]

# include both CORDEX & HighResMIP for attribution of historic trends
pr_attr <- rbind(res_cordex[,grepl("attr_PR", colnames(res_cordex))], res_hiresmip[,grepl("attr_PR", colnames(res_hiresmip))])[include,]
dI_attr <- rbind(res_cordex[,grepl("attr_DI", colnames(res_cordex))], res_hiresmip[,grepl("attr_DI", colnames(res_hiresmip))])[include,]

# projected numbers only use CORDEX
pr_proj <- res_cordex[include,grepl("proj_PR", colnames(res_cordex))]; pr_proj <- pr_proj[!is.na(pr_proj[,1]),]
dI_proj <- res_cordex[include,grepl("proj_DI", colnames(res_cordex))]; dI_proj <- dI_proj[!is.na(dI_proj[,1]),]

In [25]:
png_res <- 300

xlim_pr <- list("24" = c(0.001, 1000), "ond" = c(0.01, 10), "mam" = c(0.01, 5000))[[seas]]
xlim_dI <- list("24" = c(-50, 100), "ond" = c(-50, 150), "mam" = c(-100, 100))[[seas]]

png_height_attr <- list("24" = 400, "ond" = 500, "mam" = 800)[[seas]]
png_height_proj <- list("24" = 275, "ond" = 350, "mam" = 500)[[seas]]

png(paste0("res/synth-fig_pr-",seas,"_attr.png"), height = png_height_attr, width = png_res * 2.5); {
    prep_window(c(1,2), mar = c(3.5,1,1,1), oma = c(0,15.5,2.1,0))

    plot_synthesis(synthesis(obs_in = pr_obs, models_in = pr_attr, synth_type = "PR"), xlim = xlim_pr)
    mtext(side = 1, "Probability ratio", line = 2.5)
    plot_synthesis(synthesis(obs_in = dI_obs, models_in = dI_attr, synth_type = "rel"), hide_labels = T, xlim = xlim_dI)
    mtext(side = 1, "% change in intensity", line = 2.5)

    mtext("(a) Probability ratio (left) and intensity change (right) for\ncurrent vs 1.2°C cooler climate", side = 3, outer = T, font = 2, line = -0.5)
}; dev.off()

png(paste0("res/synth-fig_pr-",seas,"_proj.png"), height = png_height_proj, width = png_res * 2.5); {
    prep_window(c(1,2), mar = c(3.5,1,1,1), oma = c(0,15.5,2.1,0))

    plot_synthesis(synthesis(obs_in = NA, models_in = pr_proj, synth_type = "PR"), xlim = xlim_pr)
    mtext(side = 1, "Probability ratio", line = 2.5)
    plot_synthesis(synthesis(obs_in = NA, models_in = dI_proj, synth_type = "rel"), hide_labels = T, xlim = xlim_dI)
    mtext(side = 1, "% change in intensity", line = 2.5)

    mtext("(b) Probability ratio (left) and intensity change (right) for\ncurrent vs 0.8°C warmer climate", side = 3, outer = T, font = 2, line = -0.5)
}; dev.off()

agg_record_105512005 
                   2

agg_record_105512005 
                   2